In [1]:
# !pip install gensim

In [2]:
# !pip install razdel

In [3]:
# !pip install pymorphy2

In [4]:
# !pip install nltk

### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

In [5]:
from gensim.corpora.dictionary import Dictionary

import re

import numpy as np
import pandas as pd

import nltk

from nltk.corpus import stopwords

from razdel import tokenize

import pymorphy2

In [6]:
news = pd.read_csv('../lesson_2/lection2_2021_03_04/articles.csv')
news.head(3)

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [7]:
news.shape

(27000, 2)

In [8]:
users = pd.read_csv('../lesson_2/lection2_2021_03_04/users_articles.csv')
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [9]:
users.shape

(8000, 2)

In [10]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [11]:
stopword_ru = stopwords.words('russian')

In [12]:
len(stopword_ru)

151

In [13]:
morph = pymorphy2.MorphAnalyzer()

In [14]:
with open('../lesson_2/lection2_2021_03_04/stopwords.txt') as f_txt:
    additional_stopwords = [w.strip() for w in f_txt.readlines() if w]

stopword_ru += additional_stopwords

In [15]:
def clean_text(text):
    """
    Очистка текста
    """
    
    if not isinstance(text, str):
        text = str(text)
        
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    
    text = re.sub('-\s\r\n\|-\s\r\n|\r\n', '', str(text))
    text = re.sub('[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|', '', text)
    
    text = re.sub(r'\r\n\t|\n|\\s|\r\t|\\n', ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    return text

In [17]:
cache = {}

def lemmatization(text):
    """
    Лемматизация
    """
    
    # Проверка на str, если тип не сторка то меняем на строку
    if not isinstance(text, str):
        text = str(text)
        
    # Токенизация текста с помощью библиотеки razdel
    tokens = list(tokenzie(text))
    words = [_.text for _ in tokens]
    
    words_lem = []
    
    for word in words:
        if word[0] == '-':
            word = word[1:]
        
        if len(word) > 1:
            if word in cache:
                words_lem.append(cache[word])
            else:
                temp_cach = cache[w] = morph.parse(word)[0].norm_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru]
    
    return words_lem_without_stopwords
    

2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

3. Повторить п.2, но используя уже не медиану, а max

4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных